In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
import os

from langchain_community.vectorstores import SupabaseVectorStore
from supabase.client import Client, create_client

supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

E0000 00:00:1759553277.472577 1182373 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.
E0000 00:00:1759553277.509995 1182373 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


In [4]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter

loader = TextLoader("../../data/dummy.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

Created a chunk of size 510, which is longer than the specified 500
Created a chunk of size 896, which is longer than the specified 500


In [ ]:
vector_store = SupabaseVectorStore.from_documents(
    docs,
    embeddings,
    client=supabase,
    table_name="documents_temp",
    query_name="match_documents",
    chunk_size=500,
)

In [ ]:
query = "How to implement SupabaseVectorStore in langchain?"
matched_docs = vector_store.similarity_search(query)

In [7]:
print(matched_docs[0].page_content)

from langchain_community.vectorstores import SupabaseVectorStore
from langchain_openai import OpenAIEmbeddings
from supabase.client import Client, create_client

supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_SERVICE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

embeddings = OpenAIEmbeddings()

Next we'll load and parse some data for our vector store (skip if you already have documents with embeddings stored in your DB).


In [8]:
matched_docs = vector_store.similarity_search_with_relevance_scores(query)

In [9]:
matched_docs[0]

(Document(metadata={'source': '../../data/dummy.txt'}, page_content='from langchain_community.vectorstores import SupabaseVectorStore\nfrom langchain_openai import OpenAIEmbeddings\nfrom supabase.client import Client, create_client\n\nsupabase_url = os.environ.get("SUPABASE_URL")\nsupabase_key = os.environ.get("SUPABASE_SERVICE_KEY")\nsupabase: Client = create_client(supabase_url, supabase_key)\n\nembeddings = OpenAIEmbeddings()\n\nNext we\'ll load and parse some data for our vector store (skip if you already have documents with embeddings stored in your DB).'),
 0.911848967561228)

In [15]:
retriever = vector_store.as_retriever()

In [16]:
matched_docs = retriever.invoke(query)

In [17]:
for i, d in enumerate(matched_docs):
    print(f"\n## Document {i}\n")
    print(d.page_content)


## Document 0

from langchain_community.vectorstores import SupabaseVectorStore
from langchain_openai import OpenAIEmbeddings
from supabase.client import Client, create_client

supabase_url = os.environ.get("SUPABASE_URL")
supabase_key = os.environ.get("SUPABASE_SERVICE_KEY")
supabase: Client = create_client(supabase_url, supabase_key)

embeddings = OpenAIEmbeddings()

Next we'll load and parse some data for our vector store (skip if you already have documents with embeddings stored in your DB).

## Document 1

Alternatively if you already have documents with embeddings in your database, simply instantiate a new SupabaseVectorStore directly:

vector_store = SupabaseVectorStore(
    embedding=embeddings,
    client=supabase,
    table_name="documents",
    query_name="match_documents",
)

Finally, test it out by performing a similarity search:

query = "What did the president say about Ketanji Brown Jackson"
matched_docs = vector_store.similarity_search(query)

print(matched_docs[0].pa